In [ ]:
# !pip install slack_sdk

In [ ]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from datetime import datetime, timedelta
import time
from slack_mcp.utils import get_favourite_channel_ids, get_matches_for_name, compute_channelid_to_name_cached
from slack_mcp.models import NamesMatchResponse

token = os.getenv("SLACK_TOKEN")
d_cookie = os.getenv("SLACK_D_COOKIE")

In [ ]:
headers = {
    'Cookie': f'd={d_cookie}',
    'User-Agent': 'Mozilla/5.0 (compatible; Python)'
}   
client = WebClient(token=token,headers=headers)

# Write messages to db

In [ ]:
from slack_mcp.overview_utils import get_my_active_channels_from_search, get_channel_messages_with_thread_tree
from slack_mcp.utils import get_single_message

from slack_mcp.db import SLACK_MCP_DB_PATH

In [ ]:
# SLACK_MCP_DB_PATH.unlink()

In [ ]:
from slack_mcp.background_worker import run_slack_mesage_dump_background_worker_single
from slack_mcp.db import _get_slack_connection
from slack_mcp.overview_utils import get_my_active_channels_from_search

conn = _get_slack_connection()


In [ ]:
from slack_mcp.background_worker import get_timestamp_batches

In [ ]:
now = time.time()
min_ts = time.time() - 30*24*60*60 # one month ago

In [ ]:
run_slack_mesage_dump_background_worker_single(conn,
                                               client,
                                               min_ts,
                                               min_batch_length=2* 60 * 60)

In [ ]:
%debug

In [ ]:
from slack_mcp.db import get_messages_without_embeddings

get_messages_without_embeddings(conn, limit=10)

In [ ]:
# batch_start, batch_end = next(iter(
#     [x for x in get_timestamp_batches(min_ts, now, batch_days=30)])
# )

# print(batch_start, batch_end)

In [ ]:
# date_str_start = datetime.fromtimestamp(batch_start).strftime("%Y-%m-%d")
# date_str_end = datetime.fromtimestamp(batch_end).strftime("%Y-%m-%d")

In [ ]:
# date_str_start

In [ ]:
# date_str_end

In [ ]:
# from slack_mcp.utils import compute_channelid_to_name_cached

# channel_to_name, user_id_to_real_name = compute_channelid_to_name_cached(client)


In [ ]:
# channel_to_name

In [ ]:
# channel_to_name["C096WNAQDJR"]

# Get channels

In [ ]:


channel_ids = get_my_active_channels_from_search(
    client,
    last_n_days=365,
    max_pages=100,
)

print(channel_ids)

In [ ]:
channel_id = channel_ids[0]

In [ ]:
channel_id = "C093365V5UH" # inbound
ts = '1752510526.154739'
get_single_message(client, channel_id, ts)

In [ ]:
# ts = '1752510418.959799'

In [ ]:
from datetime import datetime, timedelta
import time

one_month_ago = datetime.now() - timedelta(days=30)
oldest_ts = time.mktime(one_month_ago.timetuple())


# Get all messages batched

In [ ]:
# from tqdm import tqdm

# all_messages_as_tree = []
# for channel in tqdm(channel_ids):
#     message_tree = get_channel_messages_with_thread_tree(client, channel, max_pages=100, oldest=oldest_ts)
#     all_messages_as_tree.extend(message_tree)

# # all_messages_as_tree

In [ ]:
# SLACK_MCP_DB_PATH.unlink()

In [ ]:
create_tables(conn)

In [ ]:



for msg in get_messages_without_embeddings(conn):
    print(msg)

In [ ]:
from pydantic import BaseModel
import json




In [ ]:
create_tables(conn)

In [ ]:
# for msg in all_messaages_flat:
#     if "user" not in msg:
#         continue
#     msg = get_message(conn, msg["channel_id"], msg["ts"])
#     # print(msg)

In [ ]:
all_messaages_flat[12]

In [ ]:
len(all_messages_as_tree)

In [ ]:
n_total = 0
for message in all_messages_as_tree:
    n_total += 1
    if message["replies"] is not None:
        n_total += len(message["replies"])
n_total

In [ ]:
len(all_messages_as_tree)

In [ ]:
favourite_channel_ids = get_favourite_channel_ids(client)
channel_to_name, userid2name = compute_channelid_to_name_cached(client)

In [ ]:
channel = channels[1]

In [ ]:
def get_channel_id_with_name(client, name):
    channel_to_name, userid2name = compute_channelid_to_name_cached(client)
    for channel_id, channel_name in channel_to_name.items():
        if channel_name == name:
            return channel_id
    return None

channel_id = get_channel_id_with_name(client, "Andrew Trask")
channel_id

In [ ]:
# channel_to_name[channel]

In [ ]:
message_tree = get_channel_messages_with_thread_tree(client, channel, max_pages=100)

In [ ]:
[channel_to_name[channel_id] for channel_id in channels]

In [ ]:

query = "eelco"
favourite_channel_ids = get_favourite_channel_ids(client)
channel_to_name, userid2name = compute_channelid_to_name_cached(client)
favourite_names = [channel_to_name[x] for x in favourite_channel_ids]
name2channelid = {v: k for k, v in channel_to_name.items()}
all_names = list(name2channelid.keys())
result: NamesMatchResponse = get_matches_for_name(
    query, favourite_names, all_names, name2channelid
)

In [ ]:
first_match = result.matches_in_favourites[0]
first_match

In [ ]:
channel_id = first_match.channel_id

In [ ]:
messages = client.conversations_history(channel=channel_id, limit=100)["messages"]

In [ ]:
favourite_channel_ids = get_favourite_channel_ids(client)
favourite_names = [channel_to_name[x] for x in favourite_channel_ids]

In [ ]:
# response = client.chat_postMessage(
#     channel="D7GH378QG",
#     text="test message from python slack bot"
# )
# print(response)

# Get Latest messages

In [ ]:
from slack_mcp.overview_utils import get_last_week_messages_with_threads_with_names

messages = get_last_week_messages_with_threads_with_names(client,
                                                          favourite_channel_ids,
                                                          channel_to_name,
                                                          userid2name)

In [ ]:
i=0
for k, v in messages.items():
    i+=1
    if i==3:
        print(channel_to_name[k])
        for msg in v:
            print(userid2name[msg["user"]], msg["text"])

In [ ]:
len(list(messages.values())[0])

In [ ]:
import time
time.time()

In [ ]:
n_total = 0
for c, messages in c2m.items():
    n_total += len(messages)
print(len(c2m),n_total)

In [ ]:
favourite_channel_ids

In [ ]:
float(client.conversations_history(channel="C095U8548RW", limit=10)["messages"][1]["ts"]) > last_week_timestamp

In [ ]:
def list_messages(channel_id):
    res = client.conversations_history(channel=channel_id, limit=5)
    print(channel_id_to_name[channel_id])
    for msg in res["messages"]:
        user_id = msg.get('user', 'unknown user')
        user_name = user_id_to_real_name.get(user_id, "Unknown User")
        print(f"@{user_name}: {msg['text']}")

list_messages(starred_channels[15])

In [ ]:
len(starred_channels)

In [ ]:
from slack_mcp.utils import get_all_channels

cs = get_all_channels(client)

print(len([c for c in cs if c.get("is_member", False)]))

In [ ]:
channel_ids = get_my_active_channels_from_search(client, last_n_days=7, max_pages=50)

In [ ]:
for c in channel_ids:
    print(channel_to_name[c])

In [ ]:
client.conversations_info(channel="C69RB18LA")

In [ ]:
cs[0]

In [ ]:
starred_channels = get_starred_channel_ids(client)

In [ ]:
assert len(starred_channels) == len(set(starred_channels).intersection(channel_id_to_name.keys()))

In [ ]:
from collections import defaultdict

In [ ]:
unread_messages = defaultdict(list)
for channel in starred_channels:
    
    res = client.conversations_history(channel=channel, limit=100)
    for msg in res["messages"]:
        if float(msg["ts"]) > last_week_timestamp:
            if msg.get("subtype") == "channel_join":
                continue
            
            user_id = msg.get("user", None)
            sender_name = user_id_to_real_name.get(user_id, "Unknown User")
            channel_name = channel_id_to_name.get(channel, None)
            if channel_name is None:
                channel_name = "unknown channel"
            
            
            unread_messages[channel_name].append({"user": sender_name, "text": msg["text"]})

    # print(unread_messages)
    

In [ ]:
for channel_name, msgs in unread_messages.items():
    print(f"in #{channel_name}")
    for msg in reversed(msgs):    
        print(f"@{msg['user']}: {msg['text']}")
    print("="*100)

In [ ]:
res["messages"]

## Appendix

In [ ]:
# channel_id_to_user_real_name

In [ ]:
# Format the date 7 days ago in YYYY-MM-DD
seven_days_ago_date = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")

# Search query: messages mentioning your user ID, after the date
query = f"@{my_user_ida}"


# Call search.messages API
result = client.search_messages(query=query, sort="timestamp", sort_dir="desc", count=100)




In [ ]:
for x in result["messages"]["matches"]:
    print(x)

In [ ]:
messages = result["messages"]["matches"]
for msg in messages:
    user = msg.get("user")
    text = msg.get("text")
    ts = msg.get("ts")
    channel = msg.get("channel", {}).get("name")
    print(f"[{ts}] in #{channel} by {user}: {text}")

In [ ]:
# from pprint import pprint

# # response = client.conversations_list(types="im")

# channels = [channel for channel in response["channels"] if channel["is_im"]]

# for channel in channels[:2]:
#     conversation_history = client.conversations_history(
#         channel=channel["id"],
#         limit=3,
#     )
#     print("USER", id2user[channel["id"]])
#     pprint(conversation_history["messages"])

In [ ]:
def get_all_messages(client, since_timestamp):
    response = client.conversations_history(
        channel="C06650000000000000000000",
        oldest=since_timestamp,
        limit=100,
    )
    return response["messages"]


import time
last_week_timestamp = int(time.time()) - 60 * 60 * 24 * 7
client = WebClient(token=token, headers=headers)
get_all_messages(client, last_week_timestamp)

In [ ]:
from pprint import pprint

res = []
for message in response["messages"]:
    res.append({
        "text": message["text"],
        "user": "Madhava (we're hiring)",
        "ts": message["ts"],
    })

pprint(res)

In [ ]:
response["channels"][:3]